# 데이터 베이스 생성

In [3]:
import pymysql
#데이터 베이스 생성
conn = pymysql.connect(host='localhost', user='root', password='tjdgjs789', charset='utf8')
try:
    with conn.cursor() as cursor:
        sql = 'CREATE DATABASE KU_SMART_KIOSK'
        cursor.execute(sql)
    conn.commit()
finally:
    conn.close()

# 테이블 생성

In [53]:
#테이블 생성
conn = pymysql.connect(host='localhost',
        user='root',
        password='tjdgjs789',
        db='KU_SMART_KIOSK',
        charset='utf8')
 
try:
    with conn.cursor() as cursor:
        sql = '''
            CREATE TABLE User (
                id varchar(11) NOT NULL PRIMARY KEY,
                name varchar(255) NOT NULL,
                phone varchar(255) NOT NULL,
                love varchar(255) NOT NULL,
                hate varchar(255) NOT NULL,
                blind varchar(11),
                recent varchar(255)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8
'''
        cursor.execute(sql)
    conn.commit()
finally:
    conn.close()

# 테이블에 Insert

In [5]:
#Insert 하기
def insert_sql(face_id,name,phone,like,hate,blind):
    conn = pymysql.connect(host='localhost',
            user='seongheon',
            password='tjdgjs789',
            db='KU_SMART_KIOSK',
            charset='utf8')
    display(phone)

    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO User (id,name,phone,love,hate,blind) VALUES (%s, %s, %s,%s,%s,%s)'
            cursor.execute(sql,(face_id,name,phone,like,hate,blind))
        conn.commit()
    finally:
        conn.close()

# 얼굴 데이터 수집

In [1]:
import cv2
import os
import numpy as np

def select_data(Filename):
    FilePath = 'C:\\Users\\rlat3\\임베디드 컴퓨팅\\upload\\'+Filename
    cam = cv2.VideoCapture(FilePath)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    # For each person, enter one numeric face id
    #id 지정
    f = open("count.txt",'r')
    line = f.readline()
    print(line)
    face_id = int(line) + 1
    f.close()
    f = open("count.txt",'w')
    f.write(str(face_id))
    f.close()

    # Initialize individual sampling face count
    count = 0
    while(True):
        ret, img = cam.read()
        img = cv2.flip(img, 1) # flip video image vertically
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            try:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
                count += 1
                # Save the captured image into the datasets folder
                cv2.imwrite(r"dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
                cv2.imshow('image', img)
            except Exception:
                print(Exception)
                break
        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 30: # Take 30 face sample and stop video
            break
# Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()

# 얼굴 데이터 학습

In [2]:
from PIL import Image

path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')
        print(imagePath)
        print(os.path.split(imagePath)[-1].split(".")[1])
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids
def learning():
    
    print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
    faces,ids = getImagesAndLabels(path)
    recognizer.train(faces, np.array(ids))

    # Save the model into trainer/trainer.yml
    recognizer.write('trainer/trainer.yml') # recognizer.save() worked on Mac, but not on Pi
    # Print the numer of faces trained and end program
    print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

# 학습 데이터를 라즈베리파이로 보내기

In [7]:
import requests

def send_to_rasp():
    #보내고자 하는 파일 연다
    files = open('C:\\Users\\rlat3\\임베디드 컴퓨팅\\trainer\\trainer.yml','rb')
    #파이썬 딕셔너리 형식으로 file 설정
    upload = {'file':files}
    URL = "http://127.0.0.1:10000/learning"
    res = requests.post(URL,files = upload)
    
print(send_to_rasp())

None


# Flask Server Open

In [ ]:
from flask import Flask,request
from werkzeug import secure_filename
import os
from flask import send_from_directory
import pymysql
#Insert 하기
def insert_sql(face_id,name,phone,like,hate,blind):
    conn = pymysql.connect(host='localhost',
            user='seongheon',
            password='tjdgjs789',
            db='KU_SMART_KIOSK',
            charset='utf8')
    display(phone)

    try:
        with conn.cursor() as cursor:
            sql = 'INSERT INTO User_d (id,name,phone,love,hate,blind) VALUES (%s, %s, %s,%s,%s,%s)'
            cursor.execute(sql,(face_id,name,phone,like,hate,blind))
        conn.commit()
    finally:
        conn.close()
app = Flask(__name__)
host_addr = "172.16.164.27"
port_num = "8000"
@app.route("/")
def hello():
    return "hello World"
@app.route("/upload",methods = ['POST'])
def upload_file():
    f = request.files['files']
    print(f)
    f.save("C:\\Users\\rlat3\\임베디드 컴퓨팅\\upload\\"+(f.filename))
    print("저장하였습니다.")
    select_data(f.filename)
    print("데이터셋 저장 완료")
    learning()
    print("학습완료")
    #send_to_rasp()
    print("전송완료")
    return 'upload 디렉토리 -> 파일업로드 성공'
@app.route("/upload_data",methods = ['POST'])
def upload_data():
    f = open("count.txt",'r')
    line = f.readline()
    name = request.form['name']
    phone = request.form['phone']
    like = request.form['like']
    hate = request.form['hate']
    blind = request.form['blind']
    print(blind)
    print("데이터 저장 완료")
    print("sql에 저장완료")
    insert_sql(line,name,phone,like,hate,blind)
    return "data 저장 완료"
@app.route("/get_data",methods = ['GET'])
def get_file():
    return send_from_directory(directory="C:\\Users\\rlat3\\임베디드 컴퓨팅\\trainer",filename="trainer.yml")
if __name__ == "__main__":
    app.run(host = host_addr, port= port_num)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.0.3:8000/ (Press CTRL+C to quit)


<FileStorage: '2019_12_10_04_05_59.mp4' ('video/mp4')>
저장하였습니다.
0

 [INFO] Exiting Program and cleanup stuff
데이터셋 저장 완료

 [INFO] Training faces. It will take a few seconds. Wait ...
dataset\User.1.1.jpg
1
dataset\User.1.10.jpg
1
dataset\User.1.11.jpg
1
dataset\User.1.12.jpg
1
dataset\User.1.13.jpg
1
dataset\User.1.14.jpg
1
dataset\User.1.15.jpg
1
dataset\User.1.16.jpg
1
dataset\User.1.17.jpg
1
dataset\User.1.18.jpg
1
dataset\User.1.19.jpg
1
dataset\User.1.2.jpg
1
dataset\User.1.20.jpg
1
dataset\User.1.21.jpg
1
dataset\User.1.22.jpg
1
dataset\User.1.23.jpg
1
dataset\User.1.24.jpg
1
dataset\User.1.25.jpg
1
dataset\User.1.26.jpg
1
dataset\User.1.27.jpg
1
dataset\User.1.28.jpg
1
dataset\User.1.29.jpg
1
dataset\User.1.3.jpg
1
dataset\User.1.30.jpg
1
dataset\User.1.4.jpg
1
dataset\User.1.5.jpg
1
dataset\User.1.6.jpg
1
dataset\User.1.7.jpg
1
dataset\User.1.8.jpg
1
dataset\User.1.9.jpg
1


192.168.0.5 - - [10/Dec/2019 16:08:15] "POST /upload HTTP/1.1" 200 -



 [INFO] 1 faces trained. Exiting Program
학습완료
전송완료
예
데이터 저장 완료
sql에 저장완료


'01074723267'

192.168.0.5 - - [10/Dec/2019 16:08:17] "POST /upload_data HTTP/1.1" 200 -
